
# 형태소 분석 기반 토큰화의 문제
- 형태소 분석기는 작성된 알고리즘 또는 학습된 내용을 바탕으로 토큰화를 하기 때문에 오탈자나 띄어쓰기 실수, 신조어, 외래어, 고유어 등이 사용된 경우 제대로 토큰화 하지 못한다.
- 그래서 발생 할 수있는 잠재적 문제점
    - 어휘사전을 크게 만든다.
        - 같은 의미의 단어가 형태소 분석이 안되어 여러개 등록될 수있다.
        - ex) 신조어 `돈쭐` 이라는 단어를 인식 못할 경우 `"돈쭐내러", "돈쭐나", "돈쭐냄"` 등이 다 등록 될 수 있다.
    - OOV(Out Of Vocab)에 대응하기 어렵게 만든다.
        - 같은 어근의 단어가 있지만 조사등이 바뀐 신조어등을 OOV로 인식할 수있다.




> ### 어휘 사전(Vocabulary)과 Out Of Vocabulary (OOV)
> 
> - 언어 모델링에서 **어휘 사전(Vocabulary)**은 모델이 처리할 수 있는 단어(토큰)들의 집합이다.  
> - 어휘 사전은 보통 전체 데이터셋을 토큰화한 후, 각 토큰을 고유한 정수 인덱스로 매핑해 만든다.
>    - 매핑된 정수는 모델에 입력되는 텍스트 데이터를 숫자 형식으로 변환해 모델이 처리할 수 있도록 돕는다.
>    - 예시) {"I": 1, "he": 2, "you": 3, ...}
> - **Out Of Vocabulary (OOV)**
>    - 어휘 사전(Vocab): 코퍼스를 구성하는 모든 토큰의 집합.
>    - **OOV**란 어휘 사전에 포함되지 않은 토큰을 의미하며, 모델이 해당 토큰을 처리할 수 없기 때문에 일반적으로 특별한 토큰(예: `[UNK]`)으로 대체되거나 다른 방식으로 처리된다.



# Subword Tokenization(하위 단어 토큰화)

## 정의

- Subword Tokenization은 단어를 더 작은 단위(subword)로 나누어 텍스트를 토큰화하는 방식이다.  
    - subword는 하나의 단어를 구성하는 단어들을 말한다.(coworker: co, work, er)
- 주로 자주 등장하는 단어의 일부를 공통된 토큰으로 만들고, 희귀하거나 복합적인 단어는 작은 조각(subword)으로 나누어 처리한다.
- 단어 자체를 그대로 사용하기보다는 단어의 일부를 나누어 처리함으로써 새로운 단어나 미등록 단어(Out-of-Vocabulary) 문제를 줄일 수 있다.

## 장점

1. **미등록 단어 처리 가능**  
   -  새로운 단어(신조어, 속어, 고유어등)가 등장해도 미리 정의된 subword를 조합해서 표현할 수 있어 OOV 문제를 줄일 수 있다.  

2. **어휘 크기 축소**  
   - 같은 subword를 여러 단어에서 공유함으로써, 완전한 단어를 사용하는 경우보다 어휘집의 크기를 작게 유지할 수 있다.


## 종류

1. **Byte-Pair Encoding (BPE)**  
   - 자주 등장하는 문자 쌍을 반복적으로 병합해 서브워드를 생성하는 방식.
   - OpenAI의 GPT 모델에 사용된 토크나이저이다.

2. **Unigram**  
   - 빈도기반 확률모델에 따라 subword 단위를 선택하는 방식이다.  
   - BPE보다 유연하여 더 다양한 분할 결과를 얻을 수 있다.

3. **WordPiece**  
   - BPE와 유사하지만, 빈도수가 아니라, 가능성이 높은 조합(합쳐질 가능성이 높은 subword)에 기반해 subword들을 찾는다.
   - Google의 BERT 모델에 사용된 토크나이저이다.

# Byte Pair Encoding 방식

- 원래 Text data 압축을 위해 만들어진 방법으로 text 에서 많이 등장하는 두글자 쌍의 조합을 찾아 부호화하는 알고리즘이다. 
- 연속된 글자 쌍이 더 나타나지 않거나 정해진 어휘사전 크기에 도달 할 때 까지 조합을 찾아 부호화 하는 작업을 반복한다.

## text 압축 방식의 예
- 원문: abracadabra
1. AracadAra: ab -> A :=> 원문에서 가장 빈도수 많은 ab를 A(부호로 아무 글자나 사용할 수 있다.)로 치환
2. ABcadAB: ra -> B :=> 1에서 가장 빈도수가 많은 ra를 B로 치환
3. CcadC: AB -> C :=> 2에서 가장 빈도수 맣은 AB를 C로 치환한다.(치환된 글자 쌍도 변환대상에 포함된다.)

## BPE Tokenizer 방식
BPE 토크나이저는 자주 등장하는 글자 쌍을 찾아 치환하는 대신 **단어 사전**에 추가한다.

### 예)
1. 말뭉치의 토큰들의 빈도수, 어휘사전은 아래와 같을 경우
    - 빈도사전: ('low', 5), ('lower', 2), ('newest', 6), ('widest', 3)
    - 어휘사전: ['low', 'lower', 'newest', 'widest']
2. 빈도 사전내의 모든 단어들을 글자 단위로 나눈다. (Pre Tokenization)
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w']
3. 빈도 사전을 기준으로 가장 자주 등장하는 글자 쌍(byte pair)를 찾는다.  위에서는 **'e'와 's'가 총 9번으로 가장 많이 등장함**. 'e'와 's'를 'es'로 합치고 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'es'**, 't', 6), ('w', 'i', 'd', **'es'**, 't', 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**]
4. 3 번의 과정을 계속 반복한다. 빈도수가 가장 많은 'es'와 't' 쌍을 'est'로 병합하고 'est'를 어휘 사전에 추가한다.
    - 빈도사전: ('l', 'o', 'w',  5), ('l', 'o', 'w', 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**]
5. 만약 10번 반복했다고 하면 다음과 같은 빈도 사전과 어휘 사전이 생성된다.
    - 빈도 사전: (**'low'**, 5), (**'low'**, 'e', 'r', 2), ('n', 'e', 'w', **'est'**, 6), ('w', 'i', 'd', **'est'**, 3)
    - 어휘사전: ['d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'es'**, **'est'**, **'lo'**,**'low'**, **'low'**, **'ne'**, **'new'**, **'newest'**, **'wi'**, **'wid'**, **'widest'**]

- 위와 같이 어휘 사전이 만들어 지면 원래 어휘서전에 없던 것들에 대한 처리를 할 수있다.
    - ex)
        - 'newer' :=> 'new', 'e', 'r', 
        - 'lowest' :=> 'low', 'est'
        - 'wider' :=> 'wid', 'e', 'r'

In [ ]:
## comment_txt
# 토큰화

# word 토큰, id 토큰 조회

# WordPiece tokenizer

- Byte Pair Encoding 이 빈도 기반이라면 wordpiece tokenizer는 확률 기반으로 글자 쌍을 병합한다.
- 두개 글자 쌍의 빈도수를 각 개별 글자 빈도수의 곱으로 나눈 점수가 가장 높은 순서대로 글자쌍을 묶어 나간다.

$$
score = \cfrac{f(x, y)}{f(x)\cdot f(y)} 
$$

함수 f는 빈도를 나타내며 x, y는 병합하려는 하위 단어이다.

- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', 'i', 'd', 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w')
- 가장 빈도수가 높은 쌍은 'e','s'로 9번 등장한다. 이때 각 글자는 전체에서 각각 'e'는 17번, 's'는 9번 등장한다. 위 공식에 대입하면 score는 $\frac{9}{17 \times 9} \approx 0.06$ 이다.
- 'i'와 'd' 쌍은 3번만 등장하지만 전체에서 각각 'i' 3번, 'd' 3번 등장한다. 그래서 score는 $\frac{3}{3 \times 3} \approx 0.33$ 이다.
- 나타난 빈도수는 'es' 가 많치만 더 높은 score를 가지는 'id' 쌍을 병합한다.
- 빈도사전: ('l','o','w', 5), ('l','o','w', 'e', 'r', 2), ('n', 'e', 'w', 'e', 's', 't', 6), ('w', **'id'**, 'e', 's', 't', 3)
- 어휘사전: ('d', 'e', 'i', 'l', 'n', 'o', 'r', 's', 't', 'w', **'id'**)
위의 작업을 반복해 연속된 글자 쌍이 더이상 나타나지 않거나 어휘 사전 max 크기에 도달할 때 까지 학습한다.

# Unigram 방식
- 빈도 기반 확률 모델을 사용하여 효율적으로 서브워드를 선택하고, 불필요한 서브워드를 제거해 최적의 어휘 크기를 찾는 알고리즘


- **초기 어휘 집합 구성**
    - 대상 text에 모든 단어와 그 서브스트링을 포함한 어휘 집합을 생성한다. 이 어휘 집합은 나올 수있는 모든 subword들을 다 모아놓은 것이다. 
    - 예를 들어 "hug" 단어의  ["h", "u", "g", "hu", "ug", "hug"]  substring을 만든다. 이들이 subword 후보가 된다.
- **각 Subword의 빈도수 기반 확률 계산**
    -  $\cfrac{subword가\;나타난\;횟수}{전체\;빈도수}$ 로 각 subword들의 나타난 확률을 계산한다.
- **가능한 분할에 대한 확률 계산**
    - 단어를 여러 서브워드로 분할할 수 있는 경우, 각 분할에 대한 전체 확률을 계산한다.
    - 확률 계산은 $ P(subword1)\;\times \; P(subword2)\;\times\; ..$ 으로 계산한다.
    - 예를 들어 "hug" 를 분할 한다고 했을 때
        1. \["h", "u", "g"\]: $ P(h) \times P(u) \times P(g) $
        2. \["hu", "g"\]: $ P(hu) \times P(g) $

   - 각각의 확률을 계산한 후, **가장 높은 확률**을 가진 분할을 선택한다.
     - 위 예에서 만약 1의 확률이 0.01 이고 2의 확률이 0.00001 이라면 첫번째 분할이 선택된다.

- **서브워드 제거**
    - 위의 훈견과정에서 불필요한 서브워드를 제거하면서 최적의 어휘 집합을 찾아간다. 
    - 제거 대상은 빈도수가 낮거나 조합에 크게 영향을 주지 않은 subword들이다.

In [1]:
pip install korpora

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tokenizers

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 22.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Korora 데이터셋 loading
from Korpora import Korpora
corpus = Korpora.load("korean_petitions")  # 가져올 데이터셋 이름.


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : Hyunjoong Kim lovit@github
    Repository : https://github.com/lovit/petitions_archive
    References :

    청와대 국민청원 게시판의 데이터를 월별로 수집한 것입니다.
    청원은 게시판에 글을 올린 뒤, 한달 간 청원이 진행됩니다.
    수집되는 데이터는 청원종료가 된 이후의 데이터이며, 청원 내 댓글은 수집되지 않습니다.
    단 청원의 동의 개수는 수집됩니다.
    자세한 내용은 위의 repository를 참고하세요.

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `korean_petitions` is already installed at C:\Users\Playdata\Korpora\korean_petitions\petitions_2017-08
[Korpora] Corpus `korean_petitions` is already installed at C:\Users\Playdata\Korpora\korean_petitions\petitions_2017-09
[Korpora] Corpus `korean_petitions` is already install

In [3]:
petitions = corpus.get_all_texts()  #  개별 청원들을 list로 묶어서 반환
len(petitions)

433631

In [4]:
petitions[1000]

"현재 대한민국의 환경 및 식품 안전 문제에 빨간 신호등이 켜 진지 오래다. 그것은 암 환자 발생율 증가가 말해 주고 있다. 그러나 우리나라의 암환자 발생율의 추이가 공적으로 발표되어 인구에 회자된 적이 없다. 문재인 정부에서 이것을 지금 공론화 시키는 것이 좋다고 생각한다. 왜냐하면 앞으로 있을 대선기에 이 문제가 수구 야당으로 부터 이슈화될 수 있기 때문이다. 그리고 국민의 생명을 지키는 것은 집권당의 무한 책임이다. 또 이 문제에 대한 공론화가 '적폐 청산' 및 '나라다운 나라' 라는 현 정부의 이슈에도 부합하여 정치적으로 대단히 유리하다고 본다."

In [7]:
## 파일로 저장
import os
os.makedirs('data', exist_ok=True)
with open('data/petitions_corpus.txt', 'wt', encoding='utf-8') as fw:
    for petitions in petitions:
        fw.write(petitions+"\n")  # 한줄에 하나의 청원을 저장.


In [9]:
p = open('data/petitions_corpus.txt', encoding='utf-8').read()
p[:100]

"안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어 하고 있는 줄로 압니다. 정부 부처에서는 영양사의 영양'교사'화, 폭발적인 영양'교사' 채용,"

In [10]:
p[-200:]

'습니다 네 물론 근로계약서에 사인을 하지 않으면 되죠 ? 그럼 저는 일자를 잃습니다 . 어느곳이든 갈곳이 없어지게 되겠죠 국가 공휴일 연차 대체 불가능 만들어 주십시오 교사가 당당히 연차 휴가서를 작성하게 해주십시오 사용하지 않은 연차는 연차 수당을 지급 받을수 있도록 해주십시오 보육교사 노예 해방 부탁드립니다 인권과 권리를 찾아주시길 바랍니다 !!!!!\n'

In [11]:
len(p)

222421175

In [6]:
with open('data/petitions_corpus.txt', 'rt', encoding='utf-8') as fr:
          petitions = fr.read()

In [7]:
petitions[:200]

"안녕하세요. 현재 사대, 교대 등 교원양성학교들의 예비교사들이 임용절벽에 매우 힘들어 하고 있는 줄로 압니다. 정부 부처에서는 영양사의 영양'교사'화, 폭발적인 영양'교사' 채용, 기간제 교사, 영전강, 스강의 무기계약직화가 그들의 임용 절벽과는 전혀 무관한 일이라고 주장하고 있지만 조금만 생각해보면 전혀 설득력 없는 말이라고 생각합니다. 학교 수가 같고,"

# Hugging Face tokenizers 패키지 사용해 토큰화 수행
- Subword tokenization을 처리하는 다양한 패키지(라이브러리)가 있다.

## 주요 라이브러리 
- [tokenizers](https://huggingface.co/docs/tokenizers/index)
    - huggingface에서 개발한 tokenizer 라이브러리로 BPE, WordPiece, Unigram 알고리즘을 지원한다. 
    - 설치: `pip install tokenizers`
- [Sentencepiece](https://github.com/google/sentencepiece)
    - 구글에서 개발한 subword tokenizer 라이브러리로 BPE, Unigram 방식 지원.
    - 설치: `pip install sentencepiece`

> ### korpora 말뭉치
> - 다양한 한글 데이터셋을 제공하는 패키지
> - `pip install korpora`

## Hugging Face tokenizers 패키지이용

- 설치: `pip install tokenizers`
- Tokenizer 생성
    - 토큰화 알고리즘을 지정해 instance 생성.
- Trainer 생성
    - 학습 파라미터를 설정해서 instance 생성
- Tokenizer 학습
    - train() 메소드: 학습 text 파일 경로를 지정해서 학습
    - train_from_iterator() 메소드: 학습할 string들을 iterator를 통해 제공.
- https://github.com/huggingface/tokenizers



In [1]:
import time
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer



# 토크나이저 생성
# BPR() : 토큰화 방식, unk_token : 모르는 단어(OOV-단어사전에 없는 토큰)를 표시할 스페셜 토큰을 지정.
tokenizer = Tokenizer(BPE(unk_token='[UNK]'))  
# tokenizer에 Pre Tokenizer를 설정. (BPE 전에 일단 어떻게 나누고 시작할지를 지정.)
tokenizer.pre_tokenizer = Whitespace()  # 공백 기준으로 토큰화. 토큰화한 결과를 기준으로 쌍을 찾는다.

# Train 생성 -> Tokenizer 를 어떻게 학습시킬지 설정.
trainer = BpeTrainer(   # subword 방식마다 Model과 Trainer class가 있다.
    vocab_size = 10000, # 어휘사전 크기 (고유토큰 최대 개수.)
    min_frequency = 5,  # 최소 출현 횟수. 지정한 횟수(5) 이하로 출현한 쌍(pair)은 제외한다.
    special_tokens = ["[UNK]", "[PAD]"], 
    # Vocab 에 추가할 특수토큰들 지정. (학습 텍스트에서 찾는 토큰이 아니라 명시적으로 넣어주는 토큰)
    ## 이 중 unk_token에 지정한 Unknown 토큰은 반드시 지정해야한다.
    # [UNK] : 
    # [PAD] : 문장을 model 에 입력시, 토큰의 갯수를 강제하여 임의로 채워줌.
    ## 특수 토큰은 특수 목적을 가지며, "문장시작", "문장끝", "패딩처리", "문장중간비우기" 등등이 있다. 

)



In [2]:
# 학습
s = time.time()
tokenizer.train(["data/petitions_corpus.txt"], trainer=trainer) # 학습시킬 파일들의 경로, Trainer 설정.
e = time.time()
print(e-s, "초")

22.912415504455566 초


In [33]:
####  토크나이저 저장
import os 
os.makedirs("saved_models/tokenizers")


tokenizer.save("saved_models/tokenizers/petitions_bpe.json")

FileExistsError: [WinError 183] 파일이 이미 있으므로 만들 수 없습니다: 'saved_models/tokenizers'

In [4]:
#### 불러오기
load_tokenizer = Tokenizer.from_file("saved_models/tokenizers/petitions_bpe.json")

In [5]:
# 테스트 문장
sports_txt = "프리미어리그 역대 개인 최다골 기록을 보유하고 있는 시어러가 손흥민의 골 결정력을 재차 극찬했다."
petition_txt = "이 글을 쓴 이유는 다름아닌 '전안법'시행 반대를 주장하기 위해서입니다. 먼저, '전안법'은 전기용품 및 생활용품을 판매하는 업체에서 KC인증마크를 의무적으로 받는 것입니다."
comment_txt = "멋진 식사를 즐기기에 좋은 장소 - 채식 메뉴가 정말 훌륭했습니다. 당근 케이크는 아마도 내가 먹어본 디저트 중 최고였을 거예요."

In [7]:
# 토큰으로 변환
output = tokenizer.encode(sports_txt)  # 토큰화 word, 토큰화된 index(id)

type(output)

tokenizers.Encoding

In [8]:
output.tokens # 문자로된 token (word)

['프',
 '리',
 '미',
 '어',
 '리',
 '그',
 '역',
 '대',
 '개인',
 '최',
 '다',
 '골',
 '기록',
 '을',
 '보유',
 '하고',
 '있는',
 '시',
 '어',
 '러',
 '가',
 '손',
 '흥',
 '민',
 '의',
 '골',
 '결정',
 '력을',
 '재',
 '차',
 '극',
 '찬',
 '했다',
 '.']

In [ ]:
output.ids  # 정수 토큰(id/index)

[7390,
 5123,
 5330,
 6140,
 5123,
 4128,
 6180,
 4589,
 8414,
 6891,
 4563,
 4027,
 9449,
 6364,
 9442,
 8209,
 8219,
 5908,
 6140,
 4980,
 3902,
 5802,
 7638,
 5332,
 6384,
 4027,
 8940,
 8644,
 6449,
 6793,
 4129,
 6795,
 8565,
 15]

In [10]:
tokenizer.token_to_id("프")  # word ->id

7390

In [ ]:
tokenizer.id_to_token(7390)  # id->word

'프'

In [ ]:
output.offsets[0]  # 지정한 위치의(index) 토큰이 text의 어디에 있는지 반환.

(0, 1)

In [13]:
output.ids[0], output.tokens[0], sports_txt

(7390, '프', '프리미어리그 역대 개인 최다골 기록을 보유하고 있는 시어러가 손흥민의 골 결정력을 재차 극찬했다.')

In [25]:
## comment_txt
# 토큰화
output2 = tokenizer.encode(comment_txt)
# word 토큰, id 토큰 조회
print(output2.tokens)

['멋', '진', '식', '사를', '즐', '기', '기에', '좋은', '장', '소', '-', '채', '식', '메', '뉴', '가', '정말', '훌', '륭', '했습니다', '.', '당', '근', '케', '이', '크', '는', '아', '마', '도', '내가', '먹', '어', '본', '디', '저', '트', '중', '최고', '였', '을', '거', '예', '요', '.']


In [26]:
print(output2.ids)

[5199, 6640, 5909, 8338, 6621, 4153, 8419, 8531, 6442, 5799, 14, 6811, 5909, 5206, 4513, 3902, 8343, 7589, 5093, 8365, 15, 4582, 4130, 7007, 6396, 7091, 4521, 6070, 5140, 4660, 8993, 5189, 6140, 5460, 4778, 6472, 7245, 6587, 9568, 6196, 6364, 3956, 6204, 6267, 15]


In [27]:
# petition_txt
output3 = tokenizer.encode(petition_txt)
print(output3.tokens)


['이', '글을', '쓴', '이유는', '다', '름', '아닌', "'", '전', '안', '법', "'", '시행', '반대', '를', '주장', '하기', '위해서', '입니다', '.', '먼저', ',', "'", '전', '안', '법', "'", '은', '전기', '용', '품', '및', '생활', '용', '품', '을', '판매', '하는', '업체', '에서', 'K', 'C', '인', '증', '마', '크', '를', '의무', '적으로', '받는', '것입니다', '.']


In [28]:
output3.ids

[6396,
 8577,
 6044,
 9246,
 4563,
 5107,
 8323,
 8,
 6476,
 6073,
 5412,
 8,
 8656,
 8658,
 5101,
 8953,
 8301,
 8580,
 8212,
 15,
 8855,
 13,
 8,
 6476,
 6073,
 5412,
 8,
 6360,
 9353,
 6278,
 7375,
 5345,
 8437,
 6278,
 7375,
 6364,
 9118,
 8208,
 8692,
 8210,
 44,
 36,
 6400,
 6625,
 5140,
 7091,
 5101,
 8521,
 8253,
 8546,
 8299,
 15]

In [24]:
output.ids

[5199,
 6640,
 5909,
 8338,
 6621,
 4153,
 8419,
 8531,
 6442,
 5799,
 14,
 6811,
 5909,
 5206,
 4513,
 3902,
 8343,
 7589,
 5093,
 8365,
 15,
 4582,
 4130,
 7007,
 6396,
 7091,
 4521,
 6070,
 5140,
 4660,
 8993,
 5189,
 6140,
 5460,
 4778,
 6472,
 7245,
 6587,
 9568,
 6196,
 6364,
 3956,
 6204,
 6267,
 15]

In [29]:
# id 토큰 리스트 문장으로  원복
tokenizer.decode(output3.ids)

"이 글을 쓴 이유는 다 름 아닌 ' 전 안 법 ' 시행 반대 를 주장 하기 위해서 입니다 . 먼저 , ' 전 안 법 ' 은 전기 용 품 및 생활 용 품 을 판매 하는 업체 에서 K C 인 증 마 크 를 의무 적으로 받는 것입니다 ."


## WordPiece 학습

In [ ]:
import time
import os
from tokenizers import Tokenizer
from tokenizers.models import WordPiece  # , BPR
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer     # BpeTrainer


# 
tokenizer = Tokenizer(WordPiece(unk_token='[UNK]'))
tokenizer.pre_tokenizer = Whitespace()
trainer = WordPieceTrainer(
    vocab_size=10000, 
    special_tokens=["[UNK]", "[PAD]", "[SEP]"])  # [SEP] : 문장을 나누는 구분자.
    # , continuing_subword_prefix="##"

# 학습
s = time.time()
tokenizer.train(["data/petitions_corpus.txt"], trainer=trainer)
e = time.time()
print(f"학습에 걸린시간: {e-s}초")

학습에 걸린시간: 31.19713068008423초


In [ ]:
## 저장

tokenizer.save("saved_models/tokenizers/petitions_wordpiece.json")

# Tokenizer.from_file('경로')

In [36]:
output = tokenizer.encode(petition_txt)
output.tokens

['이',
 '글',
 '##을',
 '쓴',
 '이',
 '##유',
 '##는',
 '다',
 '##름',
 '##아',
 '##닌',
 "'",
 '전',
 '##안',
 '##법',
 "'",
 '시',
 '##행',
 '반',
 '##대',
 '##를',
 '주',
 '##장',
 '##하',
 '##기',
 '위',
 '##해',
 '##서',
 '##입',
 '##니',
 '##다',
 '.',
 '먼',
 '##저',
 ',',
 "'",
 '전',
 '##안',
 '##법',
 "'",
 '은',
 '전',
 '##기',
 '##용',
 '##품',
 '및',
 '생',
 '##활',
 '##용',
 '##품',
 '##을',
 '판',
 '##매',
 '##하',
 '##는',
 '업',
 '##체',
 '##에',
 '##서',
 'K',
 '##C',
 '##인',
 '##증',
 '##마',
 '##크',
 '##를',
 '의',
 '##무',
 '##적',
 '##으',
 '##로',
 '받',
 '##는',
 '것',
 '##입',
 '##니',
 '##다',
 '.']

In [ ]:
output.ids

In [38]:
tokenizer.decode(output.ids)

"이 글 ##을 쓴 이 ##유 ##는 다 ##름 ##아 ##닌 ' 전 ##안 ##법 ' 시 ##행 반 ##대 ##를 주 ##장 ##하 ##기 위 ##해 ##서 ##입 ##니 ##다 . 먼 ##저 , ' 전 ##안 ##법 ' 은 전 ##기 ##용 ##품 및 생 ##활 ##용 ##품 ##을 판 ##매 ##하 ##는 업 ##체 ##에 ##서 K ##C ##인 ##증 ##마 ##크 ##를 의 ##무 ##적 ##으 ##로 받 ##는 것 ##입 ##니 ##다 ."

## Unigram

In [40]:
import time
import os
from tokenizers import Tokenizer
from tokenizers.models import Unigram # WordPiece  # , BPR
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import UnigramTrainer # WordPieceTrainer     # BpeTrainer



tokenizer = Tokenizer(Unigram())
tokenizer.pre_tokenizer = Whitespace()
trainer = UnigramTrainer(
    vocab_size=10000, 
    special_tokens=["[UNK]", "[PAD]", "[SEP]"])  # [SEP] : 문장을 나누는 구분자.
    # , continuing_subword_prefix="##"

# 학습
s = time.time()
tokenizer.train(["data/petitions_corpus.txt"], trainer=trainer)
e = time.time()
print(f"학습에 걸린시간: {e-s}초")

학습에 걸린시간: 212.86667370796204초


In [41]:
tokenizer.save("saved_models/tokenizers/petitions_unigram.json")

In [42]:
output = tokenizer.encode(sports_txt)
output.tokens

['프',
 '리',
 '미',
 '어',
 '리',
 '그',
 '역',
 '대',
 '개인',
 '최',
 '다',
 '골',
 '기록',
 '을',
 '보유',
 '하고',
 '있는',
 '시',
 '어',
 '러',
 '가',
 '손',
 '흥',
 '민',
 '의',
 '골',
 '결정',
 '력',
 '을',
 '재',
 '차',
 '극',
 '찬',
 '했다',
 '.']

In [43]:
output.ids

[753,
 58,
 111,
 42,
 58,
 32,
 273,
 36,
 453,
 740,
 25,
 980,
 1567,
 5,
 1608,
 19,
 45,
 38,
 42,
 252,
 9,
 443,
 1902,
 318,
 7,
 980,
 687,
 322,
 5,
 95,
 101,
 607,
 1629,
 458,
 3]